In [1]:
import numpy as np
import pandas as pd
import os
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
! pip3 install -q --upgrade git+https://github.com/echong-SG/safegraph_demo_profile
from demo_profile_functions import demo_profile_functions as dpf

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from sklearn import preprocessing

In [3]:
PADATA=pd.read_csv('PADATA.csv')

In [4]:


######### read normal 30 parts data and save above
# %%time
# patterns_df1 = pd.read_csv('core_poi-geometry-patterns-part29.csv', low_memory=False)
# #print(patterns_df1.shape)
# nydata = patterns_df1.loc[patterns_df1['city'] == 'Philadelphia']
# padata1 = nydata.dropna(subset=['visitor_home_cbgs'])
# PADATA = PADATA.append(padata1,ignore_index=True) 
print(PADATA.shape)
pd.set_option('display.max_columns', 500)
PADATA.head(5)

(18023, 54)


,Unnamed: 0,placekey,parent_placekey,safegraph_brand_ids,location_name,brands,store_id,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,open_hours,category_tags,opened_on,closed_on,tracking_closed_since,websites,geometry_type,polygon_wkt,polygon_class,enclosed,phone_number,is_synthetic,includes_parking_lot,iso_country_code,wkt_area_sq_meters,date_range_start,date_range_end,raw_visit_counts,raw_visitor_counts,visits_by_day,poi_cbg,visitor_home_cbgs,visitor_home_aggregation,visitor_daytime_cbgs,visitor_country_of_origin,distance_from_home,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_month_brand,popularity_by_hour,popularity_by_day,device_type,normalized_visits_by_state_scaling,normalized_visits_by_region_naics_visits,normalized_visits_by_region_naics_visitors,normalized_visits_by_total_visits,normalized_visits_by_total_visitors
0,0,zzw-222@63s-dvw-qj9,NaN,NaN,Mack Beverage,NaN,NaN,"Beer, Wine, and Liquor Stores","Beer, Wine, and Liquor Stores",445310.0,39.966784,-75.242585,6120 Vine St,Philadelphia,PA,19139,"{ ""Mon"": [[""10:00"", ""19:30""]], ""Tue"": [[""10:00...",Alcoholic Beverages,NaN,NaN,2019-07,[],POLYGON,POLYGON ((-75.24262792499997 39.96598729500005...,SHARED_POLYGON,False,1.215474e+10,False,False,US,2594.0,2021-08-01T00:00:00-04:00,2021-09-01T00:00:00-04:00,420.0,193.0,"[16,13,21,20,13,12,15,16,17,12,18,13,15,13,14,...",4.210101e+11,"{""421010096004"":10,""421010100001"":8,""421010114...","{""42101009600"":27,""42101010000"":8,""42101009500...","{""421010096004"":9,""421010096001"":6,""4210103600...","{""US"":155}",2041.0,23.0,"{""<5"":10,""5-10"":144,""11-20"":46,""21-60"":66,""61-...","{""Walmart"":3,""Dodge"":3,""Chrysler"":3,""The Fresh...","{""Walmart"":27,""The Fresh Grocer"":25,""Speedway""...","[52,47,48,50,51,50,48,46,52,56,56,65,79,67,70,...","{""Monday"":64,""Tuesday"":72,""Wednesday"":64,""Thur...","{""android"":130,""ios"":63}",7287.889974,0.002347,0.003775,7.777127e-06,1.120323e-05
1,1,22y-222@628-p8j-bx5,NaN,SG_BRAND_1a942d1a793ebaec8744103cb992bfb9,Philly Pretzel Factory,Philly Pretzel Factory,598,Specialty Food Stores,Baked Goods Stores,445291.0,40.037955,-75.038445,7366 Frankford Ave,Philadelphia,PA,19136,"{ ""Mon"": [[""6:00"", ""18:00""]], ""Tue"": [[""6:00"",...","Bakery,Deli,Snacks",NaN,NaN,2020-12,"[""phillypretzelfactory.com""]",POLYGON,POLYGON ((-75.03831823199994 40.03785476900003...,SHARED_POLYGON,False,1.215339e+10,False,False,US,137.0,2021-08-01T00:00:00-04:00,2021-09-01T00:00:00-04:00,4.0,4.0,"[0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0...",4.210103e+11,"{""421010030021"":4}",{},"{""421010074004"":4,""421010030021"":4}","{""US"":7}",NaN,12.0,"{""<5"":0,""5-10"":1,""11-20"":2,""21-60"":1,""61-120"":...","{""7-Eleven"":25,""Wawa"":25,""Rite Aid"":25,""Dunkin...","{""Rita's Italian Ice"":25,""Walmart"":25,""PJ Whel...","[0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,0,1,0,0,0,0,0]","{""Monday"":0,""Tuesday"":1,""Wednesday"":0,""Thursda...","{""android"":4,""ios"":4}",69.408476,0.000026,0.000042,7.406787e-08,2.321912e-07
2,2,22m-222@628-pnw-k4v,NaN,NaN,Sanctuary at Saint Lazarus,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511.0,39.968721,-75.136492,102 W Girard Ave,Philadelphia,PA,19123,"{ ""Mon"": [[""0:00"", ""2:00""], [""14:00"", ""24:00""]...","American Food,Mexican Food",NaN,NaN,2019-07,[],POLYGON,"POLYGON ((-75.13643843285763 39.9687976095611,...",OWNED_POLYGON,False,1.267719e+10,False,False,US,94.0,2021-08-01T00:00:00-04:00,2021-09-01T00:00:00-04:00,15.0,13.0,"[0,1,1,0,0,0,1,0,0,0,0,1,1,1,1,1,0,0,0,1,2,0,0...",4.210101e+11,"{""420912006021"":4,""421010147002"":4}","{""42101015800"":4}","{""421010062003"":4,""421010179005"":4,""4210102130...","{""US"":10}",3130.0,61.0,"{""<5"":1,""5-10"":1,""11-20"":0,""21-60"":5,""61-120"":...","{""Speedway"":7,""DoubleTree by Hilton"":7,""Taco B...","{""Walmart"":23,""Little Caesars"":15,""Trader Joe'...","[4,6,2,1,1,1,1,1,0,0,1,2,2,2,3,1,0,2,1,0,2,2,3,3]","{""Mo

In [5]:
#####只留下需要的Columns
cols_to_keep = ['placekey','postal_code', 'popularity_by_hour', 'popularity_by_day','visitor_home_cbgs'] 
padata1 = PADATA[cols_to_keep].copy()


In [6]:
padata2 = padata1.rename(columns={"placekey":'o_placekey'})
poi_zip2 =padata2.rename(columns={"postal_code":"placekey"})

print(poi_zip2.shape)
poi_zip2.head()
poi_zip2.to_csv('for_hour.csv')

(18023, 5)


In [7]:
#####Part1 Popularity by hour

#############这两行之间是插入的popularity_by_hour (use a website to break json file)
da_hour=pd.read_csv('break_hour_data.csv')
#summs_hour =da_hour.groupby(['placekey']).sum().reset_index()
cols_to_drop = ['Unnamed: 0','o_placekey', 'popularity_by_hour', 'popularity_by_day','visitor_home_cbgs'] 
#padata1 = PADATA[cols_to_keep].copy()
pa_hour= da_hour.drop(cols_to_drop, axis=1)
pa_hour.head()


,placekey,popularity_by_hour_1,popularity_by_hour_2,popularity_by_hour_3,popularity_by_hour_4,popularity_by_hour_5,popularity_by_hour_6,popularity_by_hour_7,popularity_by_hour_8,popularity_by_hour_9,popularity_by_hour_10,popularity_by_hour_11,popularity_by_hour_12,popularity_by_hour_13,popularity_by_hour_14,popularity_by_hour_15,popularity_by_hour_16,popularity_by_hour_17,popularity_by_hour_18,popularity_by_hour_19,popularity_by_hour_20,popularity_by_hour_21,popularity_by_hour_22,popularity_by_hour_23,popularity_by_hour_24
0,19139,52,47,48,50,51,50,48,46,52,56,56,65,79,67,70,68,76,81,73,58,63,75,80,72
1,19136,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,0,1,0,0,0,0,0
2,19123,4,6,2,1,1,1,1,1,0,0,1,2,2,2,3,1,0,2,1,0,2,2,3,3
3,19116,0,0,0,0,0,0,0,0,1,1,1,1,1,2,1,1,1,2,2,0,0,0,0,0
4,19139,17,17,17,16,16,16,16,16,17,15,15,17,14,13,14,14,17,13,13,12,14,13,16,17


In [8]:
summs_hour1 = pa_hour.groupby(['placekey']).sum().reset_index()
summs_hour1['sum_hour']= summs_hour1.iloc[:, 1:25].sum(axis=1)
summs_hour1['max_hour']= summs_hour1.iloc[:, 1:25].max(axis=1)
summs_hour1


,placekey,popularity_by_hour_1,popularity_by_hour_2,popularity_by_hour_3,popularity_by_hour_4,popularity_by_hour_5,popularity_by_hour_6,popularity_by_hour_7,popularity_by_hour_8,popularity_by_hour_9,popularity_by_hour_10,popularity_by_hour_11,popularity_by_hour_12,popularity_by_hour_13,popularity_by_hour_14,popularity_by_hour_15,popularity_by_hour_16,popularity_by_hour_17,popularity_by_hour_18,popularity_by_hour_19,popularity_by_hour_20,popularity_by_hour_21,popularity_by_hour_22,popularity_by_hour_23,popularity_by_hour_24,sum_hour,max_hour
0,8102,2,2,2,2,3,3,2,2,2,4,7,4,4,4,2,3,6,1,1,1,1,2,3,2,65,7
1,13673,58,49,45,48,71,77,211,542,705,842,865,935,900,766,719,654,612,651,583,472,344,255,186,123,10713,935
2,17602,0,0,0,0,0,0,1,2,2,4,3,2,5,6,10,10,7,5,3,1,0,0,0,0,61,10
3,18045,0,0,0,0,0,0,0,4,18,17,19,26,30,27,24,15,29,33,20,14,1,0,0,0,277,33
4,19004,8,2,1,0,0,0,0,0,0,0,1,5,17,21,12,18,19,17,20,5,7,5,2,4,164,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,19195,2,2,0,0,1,1,3,2,7,7,9,10,18,17,15,7,3,3,2,3,4,5,3,1,125,18
69,19382,12,12,12,12,12,12,12,14,9,9,12,10,14,11,10,12,9,21,30,36,30,32,20,16,379,36
70,37846,42,45,47,46,88,138,185,614,602,668,726,815,801,824,825,803,444,322,259,181,159,125,73,50,8882,825
71,39350,1915,1768,1666,1659,1757,2263,4031,11317,13318,15624,17485,20363,21208,20049,18640,18679,17110,15460,11892,8873,7896,5328,3675,2383,244359,21208


In [ ]:
#####Part2 Popularity by day

#############这两行之间是插入的popularity_by_day

In [9]:
######popularity by day
padata2 = poi_zip2.rename(columns={"visitor_home_cbgs":'o_visitor_home_cbgs'})
poi_zip3 =padata2.rename(columns={"popularity_by_day":"visitor_home_cbgs"})
home_visitor_cbgs_zip = dpf.vertically_explode_json(poi_zip3)
print(home_visitor_cbgs_zip.shape)
print("total cbg visits: {0}".format(home_visitor_cbgs_zip.visitor_count.sum()))
home_visitor_cbgs_zip.head(20)


(126161, 3)
total cbg visits: 3670544


,placekey,visitor_home_cbg,visitor_count
0,19139,Monday,64
1,19139,Tuesday,72
2,19139,Wednesday,64
3,19139,Thursday,52
4,19139,Friday,46
5,19139,Saturday,48
6,19139,Sunday,74
7,19136,Monday,0
8,19136,Tuesday,1
9,19136,Wednesday,0


In [10]:
######要代入这一步之需要选择相应的day key_'Monday', 然后算一个ratio加在最后就好了,首先汇总的是一个月的,那这样的话就看是哪个月再除以比如Monday
Weekdays = ['Monday','Tuesday','Wednesday','Thursday','Friday']
day_pattern = home_visitor_cbgs_zip.loc[home_visitor_cbgs_zip['visitor_home_cbg'].isin(Weekdays)]
#day_pattern = home_visitor_cbgs_zip.loc[home_visitor_cbgs_zip['visitor_home_cbg'] != ['Saturday','Sunday']]
summs_day1 = day_pattern.groupby(['placekey']).sum().reset_index()
summs_day = home_visitor_cbgs_zip.groupby(['placekey']).sum().reset_index()
summs_day =summs_day.rename(columns={"visitor_count":"visitor_count_sum"})
day_pattern = pd.merge(summs_day1,summs_day, left_on='placekey', right_on='placekey')
day_pattern['ratio_d']=(day_pattern['visitor_count']/day_pattern['visitor_count_sum'])/22
day_pattern.head()


,placekey,visitor_count,visitor_count_sum,ratio_d
0,8102,20,26,0.034965
1,13673,4140,5190,0.036259
2,17602,12,16,0.034091
3,18045,110,130,0.038462
4,19004,99,143,0.031469


In [ ]:
############以上是求 weekday ratio

#下一步就是带进去然后和state sampling研究一下,看我明天能不能在meet之前搞出来咯
#然后再看看residential的情况(residential就是用本地人口的demographic乘以schedule在家的)


In [11]:
#### total vistor from all of the census group
home_visitor_cbgs_zip = dpf.vertically_explode_json(poi_zip2)
print(home_visitor_cbgs_zip.shape)
print("total cbg visits: {0}".format(home_visitor_cbgs_zip.visitor_count.sum()))
home_visitor_cbgs_zip.head(20)


(320819, 3)
total cbg visits: 1539600


,placekey,visitor_home_cbg,visitor_count
0,19139,421010096004,10
1,19139,421010100001,8
2,19139,421010114001,7
3,19139,421010096005,6
4,19139,421010092002,6
5,19139,420454004013,6
6,19139,421010195012,5
7,19139,421010102001,5
8,19139,110010052011,5
9,19139,421010096001,5


In [13]:
df = pd.DataFrame({'a': [0, -1, 2], 'b': [-3, 2, 1]})
df[df < 0] = 0
df

,a,b
0,0,0
1,0,2
2,2,1


In [16]:
#vulnerable percentage
#census_year = 2019 # can be modified to 2018 as well
demo_codes = ['EP_POV150','EP_UNEMP','EP_NOHSDP','EP_AGE65','EP_DISABL','EP_MINRTY','EP_LIMENG','E_TOTPOP'] #e2 is non,

# census_df_hispanic = dpf.pd_read_csv_drive(dpf.get_drive_id(filename='cbg_b03.csv', census_year=census_year), drive=drive, dtype = {'census_block_group': str})
census_df = pd.read_csv('svi_interactive_map_us.csv')
census_df = census_df[['FIPS'] + demo_codes] # select only the columns we want
#census_f = census_df.rename(columns={"B01001e1":'total'})
census_df[census_df<0]=0
census_df.head()
#census_df_hispanic.head()

,FIPS,EP_POV150,EP_UNEMP,EP_NOHSDP,EP_AGE65,EP_DISABL,EP_MINRTY,EP_LIMENG,E_TOTPOP
0,1001020100,18.1,2.1,14.3,15.2,21.3,22.5,0.0,1941
1,1001020200,25.4,4.0,10.6,16.2,11.0,63.5,0.0,1757
2,1001020300,22.8,2.7,12.8,12.6,14.0,36.0,3.6,3694
3,1001020400,14.2,2.4,6.2,27.4,19.6,12.8,2.6,3539
4,1001020501,21.0,1.0,8.8,12.6,14.0,18.3,1.3,4306


In [17]:
#把原始数据也变成census tract
gis_data = home_visitor_cbgs_zip
gis_data['FIPS'] = gis_data['visitor_home_cbg'].str[:11]
gis_data


,placekey,visitor_home_cbg,visitor_count,FIPS
0,19139,421010096004,10,42101009600
1,19139,421010100001,8,42101010000
2,19139,421010114001,7,42101011400
3,19139,421010096005,6,42101009600
4,19139,421010092002,6,42101009200
...,...,...,...,...
320814,19150,421010263024,4,42101026302
320815,19150,421010096003,4,42101009600
320816,19150,481210216273,4,48121021627
320817,19150,421010207004,4,42101020700


In [18]:
gis_data.to_csv('gis_data.csv')

In [19]:
gis_data.FIPS = pd.to_numeric(gis_data.FIPS, errors='coerce').fillna(0).astype(np.int64)
visitors_join_zip = pd.merge(gis_data,census_df, left_on='FIPS', right_on='FIPS').drop('visitor_home_cbg',axis='columns')
print(visitors_join_zip.shape)
visitors_join_zip.head(5)

(288475, 11)


,placekey,visitor_count,FIPS,EP_POV150,EP_UNEMP,EP_NOHSDP,EP_AGE65,EP_DISABL,EP_MINRTY,EP_LIMENG,E_TOTPOP
0,19139,10,42101009600,35.7,5.5,13.8,11.1,13.3,100.0,0.8,4118
1,19139,6,42101009600,35.7,5.5,13.8,11.1,13.3,100.0,0.8,4118
2,19139,5,42101009600,35.7,5.5,13.8,11.1,13.3,100.0,0.8,4118
3,19139,4,42101009600,35.7,5.5,13.8,11.1,13.3,100.0,0.8,4118
4,19104,4,42101009600,35.7,5.5,13.8,11.1,13.3,100.0,0.8,4118


In [20]:
demos_zip = visitors_join_zip.copy()

for dc in demo_codes:
    demos_zip['visitor_count_'+dc] = demos_zip[dc] * demos_zip['visitor_count']/100 
    
print(demos_zip.shape)
demos_zip.head()
#demos_zip[['placekey','census_block_group', 'visitor_count', 'B03003e2_frac', 'B03003e3_frac', 
           #'visitor_count_B03003e2_D_adj', 'visitor_count_B03003e3_D_adj']].head(2)


(288475, 19)


,placekey,visitor_count,FIPS,EP_POV150,EP_UNEMP,EP_NOHSDP,EP_AGE65,EP_DISABL,EP_MINRTY,EP_LIMENG,E_TOTPOP,visitor_count_EP_POV150,visitor_count_EP_UNEMP,visitor_count_EP_NOHSDP,visitor_count_EP_AGE65,visitor_count_EP_DISABL,visitor_count_EP_MINRTY,visitor_count_EP_LIMENG,visitor_count_E_TOTPOP
0,19139,10,42101009600,35.7,5.5,13.8,11.1,13.3,100.0,0.8,4118,3.570,0.550,1.380,1.110,1.330,10.0,0.080,411.80
1,19139,6,42101009600,35.7,5.5,13.8,11.1,13.3,100.0,0.8,4118,2.142,0.330,0.828,0.666,0.798,6.0,0.048,247.08
2,19139,5,42101009600,35.7,5.5,13.8,11.1,13.3,100.0,0.8,4118,1.785,0.275,0.690,0.555,0.665,5.0,0.040,205.90
3,19139,4,42101009600,35.7,5.5,13.8,11.1,13.3,100.0,0.8,4118,1.428,0.220,0.552,0.444,0.532,4.0,0.032,164.72
4,19104,4,42101009600,35.7,5.5,13.8,11.1,13.3,100.0,0.8,4118,1.428,0.220,0.552,0.444,0.532,4.0,0.032,164.72


In [21]:
cols_to_keep = ['placekey','visitor_count', 'visitor_count_EP_POV150', 'visitor_count_EP_UNEMP','visitor_count_EP_NOHSDP',
               'visitor_count_EP_AGE65','visitor_count_EP_DISABL','visitor_count_EP_MINRTY','visitor_count_EP_LIMENG'] 
sums_temp = demos_zip[cols_to_keep].copy()
sums_temp.head()

,placekey,visitor_count,visitor_count_EP_POV150,visitor_count_EP_UNEMP,visitor_count_EP_NOHSDP,visitor_count_EP_AGE65,visitor_count_EP_DISABL,visitor_count_EP_MINRTY,visitor_count_EP_LIMENG
0,19139,10,3.570,0.550,1.380,1.110,1.330,10.0,0.080
1,19139,6,2.142,0.330,0.828,0.666,0.798,6.0,0.048
2,19139,5,1.785,0.275,0.690,0.555,0.665,5.0,0.040
3,19139,4,1.428,0.220,0.552,0.444,0.532,4.0,0.032
4,19104,4,1.428,0.220,0.552,0.444,0.532,4.0,0.032


In [22]:

summs_zip = sums_temp.groupby(['placekey']).sum().reset_index()

# calculate totals as percents
#total_zip_visits = summs_zip.visitor_count
#summs_walmart['B03003e3_percent'] = summs_walmart.visitor_count_B03003e3_D_adj / total_walmart_visits * 100
#summs_walmart['B03003e2_percent'] = summs_walmart.visitor_count_B03003e2_D_adj / total_walmart_visits * 100
summs_zip =summs_zip.rename(columns={"placekey":"postal_code"})
summs_zip.to_csv('PA_sums_zip.csv')
summs_zip

,postal_code,visitor_count,visitor_count_EP_POV150,visitor_count_EP_UNEMP,visitor_count_EP_NOHSDP,visitor_count_EP_AGE65,visitor_count_EP_DISABL,visitor_count_EP_MINRTY,visitor_count_EP_LIMENG
0,8102,12,2.368,0.528,1.116,2.096,1.916,4.608,0.308
1,13673,1151,283.964,69.746,114.573,159.270,170.974,157.228,9.575
2,17602,4,2.040,0.152,1.872,0.572,1.268,3.312,1.120
3,18045,57,7.049,2.114,4.588,8.615,4.720,18.642,1.667
4,19004,126,33.584,9.472,13.104,19.119,18.863,86.354,3.585
...,...,...,...,...,...,...,...,...,...
68,19195,52,19.564,4.960,6.632,7.660,7.840,35.364,1.288
69,19382,8,0.320,0.624,0.456,1.288,0.824,0.552,0.036
70,37846,787,161.199,45.110,107.095,164.026,118.181,122.175,8.452
71,39350,39794,14442.174,3461.845,7636.154,6999.624,8027.144,15613.365,244.660


In [29]:
summs_zip = pd.read_csv('PA_sums_zip.csv')

In [30]:
zipcode_list = pd.read_excel('zipcode_list.xlsx')

In [31]:
zipcode_list1 = zipcode_list['Zipcode']
zipcode_list2 =zipcode_list1.tolist()
zipcode_list2.sort()
#zipcode_list2 = zipcode_list1
print(len(zipcode_list2))

48


In [32]:
PA_zip_Sch_AS =summs_zip.loc[summs_zip['postal_code'].isin(zipcode_list2)]
#PA_zip_Sch_AS = summs_zip[summs_zip['postal_code'] in zipcode_list2]
print(len(PA_zip_Sch_AS))
PA_zip_Sch_AS.head()

48


,Unnamed: 0,postal_code,visitor_count,visitor_count_EP_POV150,visitor_count_EP_UNEMP,visitor_count_EP_NOHSDP,visitor_count_EP_AGE65,visitor_count_EP_DISABL,visitor_count_EP_MINRTY,visitor_count_EP_LIMENG
16,16,19102,20853,4817.796,1445.848,2167.974,3294.700,2905.860,9034.371,673.294
17,17,19103,28008,5521.812,1757.351,2430.726,4592.882,3456.685,11391.992,930.227
18,18,19104,81681,19659.555,5584.518,7539.051,12522.295,10497.224,38274.145,2398.596
19,19,19106,27845,5275.635,1656.648,2533.927,4585.385,3500.586,10698.307,965.487
20,20,19107,49379,11056.703,3392.275,5137.344,7655.651,6631.574,22626.955,1939.004


In [37]:
PA_zip_Sch_AS = PA_zip_Sch_AS.reset_index(drop=True)

PA_zip_Sch_AS.to_csv('PA_sums_zip_AS.csv')
PA_zip_Sch_AS.head()

,Unnamed: 0,postal_code,visitor_count,visitor_count_EP_POV150,visitor_count_EP_UNEMP,visitor_count_EP_NOHSDP,visitor_count_EP_AGE65,visitor_count_EP_DISABL,visitor_count_EP_MINRTY,visitor_count_EP_LIMENG
0,16,19102,20853,4817.796,1445.848,2167.974,3294.700,2905.860,9034.371,673.294
1,17,19103,28008,5521.812,1757.351,2430.726,4592.882,3456.685,11391.992,930.227
2,18,19104,81681,19659.555,5584.518,7539.051,12522.295,10497.224,38274.145,2398.596
3,19,19106,27845,5275.635,1656.648,2533.927,4585.385,3500.586,10698.307,965.487
4,20,19107,49379,11056.703,3392.275,5137.344,7655.651,6631.574,22626.955,1939.004


In [38]:
schedule_s = pd.read_excel('schedule_hours.xlsx')
schedule_s.head()

,c01,c02,c03,c004,c05,c06,c07,c08,Hour,Unnamed: 9
0,0.004938,0.905171,0.008866,0.017978,0.019031,0.001705,0.032415,0.002028,1,Poverty
1,0.008801,0.847872,0.013721,0.006527,0.025440,0.001705,0.069480,0.010157,2,NaN
2,0.007867,0.681084,0.056189,0.023564,0.060773,0.001705,0.128235,0.026635,3,NaN
3,0.012947,0.466589,0.080290,0.041970,0.107788,0.019358,0.176427,0.044385,4,NaN
4,0.014272,0.301364,0.112790,0.037339,0.173793,0.061429,0.199957,0.046503,5,NaN


In [39]:
iterable = list(range(1,25))
a_list = (iterable) * 48


In [40]:
summs_zip1 = PA_zip_Sch_AS
demos_zip_m1 = pd.concat([summs_zip1]*24, ignore_index=True)
demos_zip_m2=demos_zip_m1.sort_values(by=['postal_code'])
demos_zip_m2['Hour'] = a_list
demos_zip_m2= demos_zip_m2.reset_index()
demos_zip_m2= demos_zip_m2.drop(['index'], axis=1)
#print(demos_zip_m1.shape)
demos_zip_m2.to_csv('zip_visitor_count')
print(len(demos_zip_m2))
demos_zip_m2.head()

1152


,Unnamed: 0,postal_code,visitor_count,visitor_count_EP_POV150,visitor_count_EP_UNEMP,visitor_count_EP_NOHSDP,visitor_count_EP_AGE65,visitor_count_EP_DISABL,visitor_count_EP_MINRTY,visitor_count_EP_LIMENG,Hour
0,16,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,1
1,16,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,2
2,16,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,3
3,16,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,4
4,16,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,5


In [41]:
demos_zip_m2.iloc[:,2]

0       20853
1       20853
2       20853
3       20853
4       20853
        ...  
1147    39112
1148    39112
1149    39112
1150    39112
1151    39112
Name: visitor_count, Length: 1152, dtype: int64

In [42]:
building_type = pd.read_csv('building_type.csv')
building_type.head()

/var/folders/jv/hgb42cb56db_s0m92d8s7f040000gn/T/ipykernel_97105/585035241.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  building_type = pd.read_csv('building_type.csv')


,ID,Unnamed: 0,City,State,Zip,FIPS,Lon,Lat,Census,Census_O,Type,quality,quality_O,condition,Year,stories,rooms,Roof_M,Roof_S,heating,Air_C,fireplace,Area,Wal,Con
0,0000CF06-0000-0000-A475-000000000000,4429624,Philadelphia,PA,19140,42101,40.018284,-75.131681,4.210100e+14,4.210104e+10,RR108,NaN,NaN,NaN,1930.0,2.0,6.0,NaN,NaN,YY,NaN,NaN,992.0,BR,NaN
1,0000D006-0000-0000-A475-000000000000,4492706,Philadelphia,PA,19122,42101,39.975674,-75.138696,4.210100e+14,4.210102e+10,RR108,NaN,NaN,NaN,1920.0,2.0,6.0,NaN,NaN,NaN,NaN,NaN,788.0,BR,NaN
2,0000D106-0000-0000-A475-000000000000,4556208,Philadelphia,PA,19142,42101,39.927032,-75.242541,4.210100e+14,4.210101e+10,RI112,NaN,NaN,NaN,1950.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1514.0,BR,NaN
3,0000D306-0000-0000-A475-000000000000,4682914,Philadelphia,PA,19114,42101,40.077419,-74.988958,4.210100e+14,4.210104e+10,RI112,NaN,NaN,NaN,1920.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,3000.0,BR,NaN
4,0000D406-0000-0000-A475-000000000000,4745366,Philadelphia,PA,19121,42101,39.974883,-75.173053,4.210100e+14,4.210101e+10,RR108,NaN,NaN,NaN,1930.0,2.0,7.0,NaN,NaN,HW,NaN,NaN,1578.0,BR,NaN


In [43]:
building_type1 = building_type[['Zip','Type']]

In [44]:
building_type1['Type_1'] = building_type1['Type'].str[:2]

/var/folders/jv/hgb42cb56db_s0m92d8s7f040000gn/T/ipykernel_97105/147580275.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  building_type1['Type_1'] = building_type1['Type'].str[:2]


In [45]:
building_type1.head()

,Zip,Type,Type_1
0,19140,RR108,RR
1,19122,RR108,RR
2,19142,RI112,RI
3,19114,RI112,RI
4,19121,RR108,RR


In [46]:
building_type1.to_csv('building_type1_2digit.csv')

In [47]:
building_type1 = pd.read_csv('building_type1_2digit.csv')

In [48]:
building_dis = pd.read_excel('building_dis.xlsx')
building_dis=building_dis.drop(['ECR_c02'], axis=1)
print(len(building_dis))
#building_dis_norm = preprocessing.normalize(building_dis)
building_dis.head()

48


,zipcode,EGV_c01,ECR_c04,ECII_c05,EEI_c06,ERC_c07
0,19102,0.186441,0.140678,0.245763,0.067797,0.359322
1,19103,0.041719,0.471980,0.366127,0.086550,0.033624
2,19104,0.009669,0.157459,0.529006,0.251381,0.052486
3,19106,0.015625,0.245536,0.613839,0.055804,0.069196
4,19107,0.045012,0.291971,0.564477,0.071776,0.026764


In [49]:
schedule_s_b = pd.concat([schedule_s]*48, ignore_index=True)

In [50]:
iterable = list(range(1,25))
b_list = (iterable) * 7*48

schedule_s_abu=building_dis

schedule_bd1=pd.concat([schedule_s_abu]*24*7).sort_index()
schedule_bd1['Hour'] = b_list
schedule_bd1= schedule_bd1.reset_index(drop=True)

schedule_bd1.head()
# schedule_bd = pd.concat([schedule_s_abu]*24*7, ignore_index=True)
# schedule_bd1=schedule_bd.sort_values(by=['zipcode'])
# schedule_bd1['Hour'] = b_list
# schedule_bd1.head(30)

,zipcode,EGV_c01,ECR_c04,ECII_c05,EEI_c06,ERC_c07,Hour
0,19102,0.186441,0.140678,0.245763,0.067797,0.359322,1
1,19102,0.186441,0.140678,0.245763,0.067797,0.359322,2
2,19102,0.186441,0.140678,0.245763,0.067797,0.359322,3
3,19102,0.186441,0.140678,0.245763,0.067797,0.359322,4
4,19102,0.186441,0.140678,0.245763,0.067797,0.359322,5


In [51]:
print(len(schedule_s_b))

8064


In [52]:
#它统计的其实本来就是commercial building, 所以除非要细有的
# schedule_s_b['schedule']=(schedule_bd1['EGV_c01']*schedule_s_b['c01'])
# +(schedule_bd1['ECR_c04']*schedule_s_b['c004'])+(schedule_bd1['ECII_c05']*schedule_s_b['c05'])
# +(schedule_bd1['EEI_c06']*schedule_s_b['c06'])
# +(schedule_bd1['ERC_c07']*schedule_s_b['c07'])
cols = ['c01', 'c004', 'c05','c06','c07']

schedule_s_b['schedule1']=schedule_s_b[cols].sum(axis=1)

schedule_s_b['zipcode']=schedule_bd1['zipcode']
schedule_s_b['schedule1_try']=(schedule_bd1['EGV_c01']*schedule_s_b['c01'])
schedule_s_b.head()

,c01,c02,c03,c004,c05,c06,c07,c08,Hour,Unnamed: 9,schedule1,zipcode,schedule1_try
0,0.004938,0.905171,0.008866,0.017978,0.019031,0.001705,0.032415,0.002028,1,Poverty,0.076067,19102,0.000921
1,0.008801,0.847872,0.013721,0.006527,0.025440,0.001705,0.069480,0.010157,2,NaN,0.111952,19102,0.001641
2,0.007867,0.681084,0.056189,0.023564,0.060773,0.001705,0.128235,0.026635,3,NaN,0.222144,19102,0.001467
3,0.012947,0.466589,0.080290,0.041970,0.107788,0.019358,0.176427,0.044385,4,NaN,0.358491,19102,0.002414
4,0.014272,0.301364,0.112790,0.037339,0.173793,0.061429,0.199957,0.046503,5,NaN,0.486790,19102,0.002661


In [61]:
schedule_s_b.iloc[:24,10]

0     0.076067
1     0.111952
2     0.222144
3     0.358491
4     0.486790
5     0.578703
6     0.644118
7     0.668728
8     0.618917
9     0.652698
10    0.682633
11    0.655666
12    0.668159
13    0.591267
14    0.643602
15    0.638682
16    0.616516
17    0.520365
18    0.340931
19    0.179278
20    0.089437
21    0.051316
22    0.021800
23    0.018531
Name: schedule1, dtype: float64

In [53]:
cols_to_drop = ['Unnamed: 0'] 
#padata1 = PADATA[cols_to_keep].copy()
demos_zip_m2= demos_zip_m2.drop(cols_to_drop, axis=1)
demos_zip_m2.head()


,postal_code,visitor_count,visitor_count_EP_POV150,visitor_count_EP_UNEMP,visitor_count_EP_NOHSDP,visitor_count_EP_AGE65,visitor_count_EP_DISABL,visitor_count_EP_MINRTY,visitor_count_EP_LIMENG,Hour
0,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,1
1,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,2
2,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,3
3,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,4
4,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,5


In [62]:
final_list1= pd.DataFrame(columns=['postal_code','visitor_count','visitor_count_EP_POV150','visitor_count_EP_UNEMP','visitor_count_EP_NOHSDP','visitor_count_EP_AGE65','visitor_count_EP_DISABL',
                                          'visitor_count_EP_MINRTY','visitor_count_EP_LIMENG','Hour','poverty','unemployed','education','aged','disabled','minority','language'])
for zips in zipcode_list2:
    schedule_zip = schedule_s_b.loc[schedule_s_b['zipcode']==zips]
    demos_zip = demos_zip_m2.loc[demos_zip_m2['postal_code']==zips]
    poverty_s1 = (schedule_zip.iloc[:24,10].values.tolist())
    unemployed_s1 = (schedule_zip.iloc[24:48,10].values.tolist())
    education_s1 = (schedule_zip.iloc[48:72,10].values.tolist())
    aged_s1 = (schedule_zip.iloc[72:96,10].values.tolist())
    disabled_s1 = (schedule_zip.iloc[96:120,10].values.tolist())
    minority_s1 = (schedule_zip.iloc[120:144,10].values.tolist())
    language_s1 = (schedule_zip.iloc[144:168,10].values.tolist())
    demos_zip['poverty'] = demos_zip.iloc[:,2]*poverty_s1
    demos_zip['unemployed'] = demos_zip.iloc[:,3]*unemployed_s1
    demos_zip['education'] = demos_zip.iloc[:,4]*education_s1
    demos_zip['aged'] = demos_zip.iloc[:,5]*aged_s1
    demos_zip['disabled'] = demos_zip.iloc[:,6]*disabled_s1
    demos_zip['minority'] = demos_zip.iloc[:,7]*minority_s1
    demos_zip['language'] = demos_zip.iloc[:,8]*language_s1
    final_list1= final_list1.append(demos_zip,ignore_index=True) 


/var/folders/jv/hgb42cb56db_s0m92d8s7f040000gn/T/ipykernel_97105/1220967176.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demos_zip['poverty'] = demos_zip.iloc[:,2]*poverty_s1
/var/folders/jv/hgb42cb56db_s0m92d8s7f040000gn/T/ipykernel_97105/1220967176.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demos_zip['unemployed'] = demos_zip.iloc[:,3]*unemployed_s1
/var/folders/jv/hgb42cb56db_s0m92d8s7f040000gn/T/ipykernel_97105/1220967176.py:15: SettingWithCopyWarning: 
A value is trying to be set on a

In [63]:
final_list1.head()


,postal_code,visitor_count,visitor_count_EP_POV150,visitor_count_EP_UNEMP,visitor_count_EP_NOHSDP,visitor_count_EP_AGE65,visitor_count_EP_DISABL,visitor_count_EP_MINRTY,visitor_count_EP_LIMENG,Hour,poverty,unemployed,education,aged,disabled,minority,language
0,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,1,366.473913,79.283763,175.813295,192.986245,203.463587,599.720139,18.435575
1,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,2,539.362799,116.838744,314.028864,314.777500,310.786794,1201.154728,65.774159
2,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,3,1070.245656,229.227772,582.772446,610.711860,624.227167,2219.695994,154.805481
3,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,4,1727.136016,420.557202,899.848102,1035.934565,904.084902,3932.924253,270.573487
4,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,5,2345.256836,567.525249,1137.192351,1350.609113,1228.436330,5271.963494,392.473580


In [64]:
final_list1.to_csv('final_distribution_update.csv')


In [65]:
final_list1 = pd.read_csv('final_distribution_update.csv')

In [66]:
day_pattern.head()


,placekey,visitor_count,visitor_count_sum,ratio_d
0,8102,20,26,0.034965
1,13673,4140,5190,0.036259
2,17602,12,16,0.034091
3,18045,110,130,0.038462
4,19004,99,143,0.031469


In [67]:
day_pattern.head()
# day_pattern=day_pattern.loc[day_pattern['placekey']].isin(zipcode_list2)
day_pattern1 =day_pattern.loc[day_pattern['placekey'].isin(zipcode_list2)]
day_pattern1 = day_pattern1.reset_index(drop=True)
day_pattern2 = pd.concat([day_pattern1]*24, ignore_index=True)
day_pattern2=day_pattern2.sort_values(by=['placekey'])
day_pattern2 = day_pattern2.reset_index(drop=True)
day_pattern2.head()
print(len(day_pattern2))


1152


In [68]:
final_list1['ratio_wday'] = day_pattern2['ratio_d']


In [69]:
final_list1.head()

,Unnamed: 0,postal_code,visitor_count,visitor_count_EP_POV150,visitor_count_EP_UNEMP,visitor_count_EP_NOHSDP,visitor_count_EP_AGE65,visitor_count_EP_DISABL,visitor_count_EP_MINRTY,visitor_count_EP_LIMENG,Hour,poverty,unemployed,education,aged,disabled,minority,language,ratio_wday
0,0,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,1,366.473913,79.283763,175.813295,192.986245,203.463587,599.720139,18.435575,0.033301
1,1,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,2,539.362799,116.838744,314.028864,314.777500,310.786794,1201.154728,65.774159,0.033301
2,2,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,3,1070.245656,229.227772,582.772446,610.711860,624.227167,2219.695994,154.805481,0.033301
3,3,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,4,1727.136016,420.557202,899.848102,1035.934565,904.084902,3932.924253,270.573487,0.033301
4,4,19102,20853,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,5,2345.256836,567.525249,1137.192351,1350.609113,1228.436330,5271.963494,392.473580,0.033301


In [70]:
summs_hour1.head()

,placekey,popularity_by_hour_1,popularity_by_hour_2,popularity_by_hour_3,popularity_by_hour_4,popularity_by_hour_5,popularity_by_hour_6,popularity_by_hour_7,popularity_by_hour_8,popularity_by_hour_9,popularity_by_hour_10,popularity_by_hour_11,popularity_by_hour_12,popularity_by_hour_13,popularity_by_hour_14,popularity_by_hour_15,popularity_by_hour_16,popularity_by_hour_17,popularity_by_hour_18,popularity_by_hour_19,popularity_by_hour_20,popularity_by_hour_21,popularity_by_hour_22,popularity_by_hour_23,popularity_by_hour_24,sum_hour,max_hour
0,8102,2,2,2,2,3,3,2,2,2,4,7,4,4,4,2,3,6,1,1,1,1,2,3,2,65,7
1,13673,58,49,45,48,71,77,211,542,705,842,865,935,900,766,719,654,612,651,583,472,344,255,186,123,10713,935
2,17602,0,0,0,0,0,0,1,2,2,4,3,2,5,6,10,10,7,5,3,1,0,0,0,0,61,10
3,18045,0,0,0,0,0,0,0,4,18,17,19,26,30,27,24,15,29,33,20,14,1,0,0,0,277,33
4,19004,8,2,1,0,0,0,0,0,0,0,1,5,17,21,12,18,19,17,20,5,7,5,2,4,164,21


In [71]:
hour_pattern1 =summs_hour1.loc[summs_hour1['placekey'].isin(zipcode_list2)]
hour_pattern1 = hour_pattern1.reset_index(drop=True)
#hour_pattern2 = pd.concat([hour_pattern1]*24, ignore_index=True)
#hour_pattern2=hour_pattern2.sort_values(by=['placekey'])
#hour_pattern2 = hour_pattern2.reset_index(drop=True)
hour_pattern1.head()
#print(len(day_pattern2))
#### popularity 要在sample完之后再加上去,因为它有累计

,placekey,popularity_by_hour_1,popularity_by_hour_2,popularity_by_hour_3,popularity_by_hour_4,popularity_by_hour_5,popularity_by_hour_6,popularity_by_hour_7,popularity_by_hour_8,popularity_by_hour_9,popularity_by_hour_10,popularity_by_hour_11,popularity_by_hour_12,popularity_by_hour_13,popularity_by_hour_14,popularity_by_hour_15,popularity_by_hour_16,popularity_by_hour_17,popularity_by_hour_18,popularity_by_hour_19,popularity_by_hour_20,popularity_by_hour_21,popularity_by_hour_22,popularity_by_hour_23,popularity_by_hour_24,sum_hour,max_hour
0,19102,13122,13377,13232,13145,13172,13448,14539,16492,18975,20519,21545,22181,22366,21932,21027,18851,16407,13774,11537,7961,8693,11519,12581,13290,373685,22366
1,19103,6403,6173,5950,5868,5895,6387,7661,9914,12877,14875,15756,16351,16669,16571,15955,14619,13045,11190,9555,7940,8113,9042,8395,7503,252707,16669
2,19104,29252,28646,28272,27992,28633,32162,40714,51413,62127,71690,76965,79951,81143,79200,74120,64905,54473,44879,39082,31298,30175,35011,35158,33104,1160365,81143
3,19106,4400,3977,3750,3548,3558,3853,4839,6290,8145,9576,10745,11607,12350,12694,12328,11548,10191,8338,7482,6310,6905,7723,6929,5771,182857,12694
4,19107,10145,9427,8770,8271,8266,8796,11021,14036,17758,20357,21300,22180,23082,23430,23381,22247,20356,17548,15629,12767,12434,13704,13141,12043,370089,23430


In [72]:
########在这一步可以改max_hour
for i in range (24):
    i = i +1
    t = str(i)
    hour_pattern1['popularity_by_hour_'+t]=hour_pattern1['popularity_by_hour_'+t]/hour_pattern1['max_hour']


In [73]:
hour_pattern1.head()

,placekey,popularity_by_hour_1,popularity_by_hour_2,popularity_by_hour_3,popularity_by_hour_4,popularity_by_hour_5,popularity_by_hour_6,popularity_by_hour_7,popularity_by_hour_8,popularity_by_hour_9,popularity_by_hour_10,popularity_by_hour_11,popularity_by_hour_12,popularity_by_hour_13,popularity_by_hour_14,popularity_by_hour_15,popularity_by_hour_16,popularity_by_hour_17,popularity_by_hour_18,popularity_by_hour_19,popularity_by_hour_20,popularity_by_hour_21,popularity_by_hour_22,popularity_by_hour_23,popularity_by_hour_24,sum_hour,max_hour
0,19102,0.586694,0.598095,0.591612,0.587722,0.588930,0.601270,0.650049,0.737369,0.848386,0.917419,0.963292,0.991729,1.000000,0.980596,0.940132,0.842842,0.733569,0.615845,0.515828,0.355942,0.388670,0.515023,0.562506,0.594205,373685,22366
1,19103,0.384126,0.370328,0.356950,0.352031,0.353650,0.383166,0.459596,0.594757,0.772512,0.892375,0.945228,0.980923,1.000000,0.994121,0.957166,0.877017,0.782590,0.671306,0.573220,0.476333,0.486712,0.542444,0.503629,0.450117,252707,16669
2,19104,0.360499,0.353031,0.348422,0.344971,0.352871,0.396362,0.501756,0.633610,0.765648,0.883502,0.948511,0.985310,1.000000,0.976055,0.913449,0.799884,0.671321,0.553085,0.481644,0.385714,0.371874,0.431473,0.433284,0.407971,1160365,81143
3,19106,0.346620,0.313298,0.295415,0.279502,0.280290,0.303529,0.381204,0.495510,0.641642,0.754372,0.846463,0.914369,0.972901,1.000000,0.971167,0.909721,0.802820,0.656846,0.589412,0.497085,0.543958,0.608398,0.545848,0.454624,182857,12694
4,19107,0.432992,0.402347,0.374306,0.353009,0.352796,0.375416,0.470380,0.599061,0.757917,0.868843,0.909091,0.946650,0.985147,1.000000,0.997909,0.949509,0.868801,0.748954,0.667051,0.544900,0.530687,0.584891,0.560862,0.513999,370089,23430


In [74]:
hour_pattern3 = hour_pattern1.transpose()
hour_pattern3 = hour_pattern3.drop(['placekey','sum_hour','max_hour'], axis='rows')
hour_pattern3.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
popularity_by_hour_1,0.586694,0.384126,0.360499,0.346620,0.432992,0.500000,0.501141,0.341374,0.225869,0.167374,0.282098,0.376505,0.399723,0.666752,0.851458,0.494379,0.489026,0.534220,0.578844,0.707464,0.657501,0.464772,0.427344,0.556782,0.665308,0.688907,0.575076,0.538192,0.489364,0.357240,0.236002,0.627186,0.652427,0.566318,0.428968,0.699760,0.696406,0.557358,0.419082,0.529729,0.342003,0.250313,0.491021,0.435676,0.458113,0.327009,0.383375,0.221065
popularity_by_hour_2,0.598095,0.370328,0.353031,0.313298,0.402347,0.250000,0.493180,0.340325,0.209549,0.145235,0.260347,0.367080,0.376862,0.676539,0.828989,0.492604,0.458114,0.526288,0.559685,0.709060,0.602854,0.460791,0.422962,0.561016,0.672213,0.691221,0.557508,0.525358,0.475938,0.334771,0.220782,0.614550,0.644364,0.552015,0.394066,0.684548,0.709844,0.556858,0.400399,0.518386,0.304500,0.222946,0.470542,0.399918,0.440651,0.298307,0.334174,0.211500
popularity_by_hour_3,0.591612,0.356950,0.348422,0.295415,0.374306,0.166667,0.488280,0.339801,0.199328,0.136746,0.251223,0.359923,0.373052,0.677446,0.809380,0.488594,0.418341,0.524834,0.548679,0.696906,0.540202,0.461587,0.422962,0.563134,0.670198,0.687531,0.537664,0.513527,0.464026,0.319224,0.216272,0.595278,0.629077,0.543706,0.367275,0.676312,0.705282,0.548996,0.388763,0.511800,0.281025,0.215337,0.460775,0.389231,0.432411,0.289842,0.311641,0.205067
popularity_by_hour_4,0.587722,0.352031,0.344971,0.279502,0.353009,0.166667,0.485830,0.339276,0.200986,0.133000,0.250136,0.360098,0.368202,0.679584,0.796797,0.484255,0.397115,0.523518,0.545553,0.697275,0.529760,0.460632,0.423839,0.563904,0.667595,0.682966,0.528691,0.512024,0.458258,0.308443,0.223224,0.587620,0.622533,0.541657,0.360337,0.671966,0.705530,0.551998,0.384641,0.513447,0.272453,0.209989,0.453634,0.386354,0.430842,0.279918,0.360562,0.205295
popularity_by_hour_5,0.588930,0.353650,0.352871,0.280290,0.352796,0.083333,0.489394,0.352386,0.211253,0.135664,0.260226,0.360971,0.364738,0.680468,0.790996,0.488199,0.392066,0.528089,0.540186,0.701203,0.523495,0.461747,0.439790,0.563968,0.666440,0.680778,0.526416,0.515184,0.466673,0.311394,0.230177,0.585067,0.620177,0.542226,0.357349,0.661100,0.705778,0.555571,0.390492,0.512441,0.269141,0.207483,0.450746,0.390464,0.432607,0.281183,0.521972,0.206661


In [75]:
aa = final_list1['visitor_count']
for i in range(48):
    a = i*24
    b = (i +1)*24
    aa[a:b]=hour_pattern3[i]

/var/folders/jv/hgb42cb56db_s0m92d8s7f040000gn/T/ipykernel_97105/1543659824.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa[a:b]=hour_pattern3[i]


In [76]:
aa[0:2]

0    0.586694
1    0.598095
Name: visitor_count, dtype: float64

In [77]:
final_list1['hour_ratio']=aa[0:1152]
#final_list1 = final_list1.drop(['Unnamed: 0'], axis='columns')
final_list1.head()

,Unnamed: 0,postal_code,visitor_count,visitor_count_EP_POV150,visitor_count_EP_UNEMP,visitor_count_EP_NOHSDP,visitor_count_EP_AGE65,visitor_count_EP_DISABL,visitor_count_EP_MINRTY,visitor_count_EP_LIMENG,Hour,poverty,unemployed,education,aged,disabled,minority,language,ratio_wday,hour_ratio
0,0,19102,0.586694,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,1,366.473913,79.283763,175.813295,192.986245,203.463587,599.720139,18.435575,0.033301,0.586694
1,1,19102,0.598095,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,2,539.362799,116.838744,314.028864,314.777500,310.786794,1201.154728,65.774159,0.033301,0.598095
2,2,19102,0.591612,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,3,1070.245656,229.227772,582.772446,610.711860,624.227167,2219.695994,154.805481,0.033301,0.591612
3,3,19102,0.587722,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,4,1727.136016,420.557202,899.848102,1035.934565,904.084902,3932.924253,270.573487,0.033301,0.587722
4,4,19102,0.588930,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,5,2345.256836,567.525249,1137.192351,1350.609113,1228.436330,5271.963494,392.473580,0.033301,0.588930


In [78]:
final_list1.to_csv('final_distribution_hourly_weekday_update.csv')


In [ ]:
############### now lets do sampling, 感觉按照那个,就找出sample rate, 和multifizer 乘一下

In [101]:
PADATA.head(5)

,Unnamed: 0,placekey,parent_placekey,safegraph_brand_ids,location_name,brands,store_id,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,open_hours,category_tags,opened_on,closed_on,tracking_closed_since,websites,geometry_type,polygon_wkt,polygon_class,enclosed,phone_number,is_synthetic,includes_parking_lot,iso_country_code,wkt_area_sq_meters,date_range_start,date_range_end,raw_visit_counts,raw_visitor_counts,visits_by_day,poi_cbg,visitor_home_cbgs,visitor_home_aggregation,visitor_daytime_cbgs,visitor_country_of_origin,distance_from_home,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_month_brand,popularity_by_hour,popularity_by_day,device_type,normalized_visits_by_state_scaling,normalized_visits_by_region_naics_visits,normalized_visits_by_region_naics_visitors,normalized_visits_by_total_visits,normalized_visits_by_total_visitors
0,0,zzw-222@63s-dvw-qj9,NaN,NaN,Mack Beverage,NaN,NaN,"Beer, Wine, and Liquor Stores","Beer, Wine, and Liquor Stores",445310.0,39.966784,-75.242585,6120 Vine St,Philadelphia,PA,19139,"{ ""Mon"": [[""10:00"", ""19:30""]], ""Tue"": [[""10:00...",Alcoholic Beverages,NaN,NaN,2019-07,[],POLYGON,POLYGON ((-75.24262792499997 39.96598729500005...,SHARED_POLYGON,False,1.215474e+10,False,False,US,2594.0,2021-08-01T00:00:00-04:00,2021-09-01T00:00:00-04:00,420.0,193.0,"[16,13,21,20,13,12,15,16,17,12,18,13,15,13,14,...",4.210101e+11,"{""421010096004"":10,""421010100001"":8,""421010114...","{""42101009600"":27,""42101010000"":8,""42101009500...","{""421010096004"":9,""421010096001"":6,""4210103600...","{""US"":155}",2041.0,23.0,"{""<5"":10,""5-10"":144,""11-20"":46,""21-60"":66,""61-...","{""Walmart"":3,""Dodge"":3,""Chrysler"":3,""The Fresh...","{""Walmart"":27,""The Fresh Grocer"":25,""Speedway""...","[52,47,48,50,51,50,48,46,52,56,56,65,79,67,70,...","{""Monday"":64,""Tuesday"":72,""Wednesday"":64,""Thur...","{""android"":130,""ios"":63}",7287.889974,0.002347,0.003775,7.777127e-06,1.120323e-05
1,1,22y-222@628-p8j-bx5,NaN,SG_BRAND_1a942d1a793ebaec8744103cb992bfb9,Philly Pretzel Factory,Philly Pretzel Factory,598,Specialty Food Stores,Baked Goods Stores,445291.0,40.037955,-75.038445,7366 Frankford Ave,Philadelphia,PA,19136,"{ ""Mon"": [[""6:00"", ""18:00""]], ""Tue"": [[""6:00"",...","Bakery,Deli,Snacks",NaN,NaN,2020-12,"[""phillypretzelfactory.com""]",POLYGON,POLYGON ((-75.03831823199994 40.03785476900003...,SHARED_POLYGON,False,1.215339e+10,False,False,US,137.0,2021-08-01T00:00:00-04:00,2021-09-01T00:00:00-04:00,4.0,4.0,"[0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0...",4.210103e+11,"{""421010030021"":4}",{},"{""421010074004"":4,""421010030021"":4}","{""US"":7}",NaN,12.0,"{""<5"":0,""5-10"":1,""11-20"":2,""21-60"":1,""61-120"":...","{""7-Eleven"":25,""Wawa"":25,""Rite Aid"":25,""Dunkin...","{""Rita's Italian Ice"":25,""Walmart"":25,""PJ Whel...","[0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,0,1,0,0,0,0,0]","{""Monday"":0,""Tuesday"":1,""Wednesday"":0,""Thursda...","{""android"":4,""ios"":4}",69.408476,0.000026,0.000042,7.406787e-08,2.321912e-07
2,2,22m-222@628-pnw-k4v,NaN,NaN,Sanctuary at Saint Lazarus,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511.0,39.968721,-75.136492,102 W Girard Ave,Philadelphia,PA,19123,"{ ""Mon"": [[""0:00"", ""2:00""], [""14:00"", ""24:00""]...","American Food,Mexican Food",NaN,NaN,2019-07,[],POLYGON,"POLYGON ((-75.13643843285763 39.9687976095611,...",OWNED_POLYGON,False,1.267719e+10,False,False,US,94.0,2021-08-01T00:00:00-04:00,2021-09-01T00:00:00-04:00,15.0,13.0,"[0,1,1,0,0,0,1,0,0,0,0,1,1,1,1,1,0,0,0,1,2,0,0...",4.210101e+11,"{""420912006021"":4,""421010147002"":4}","{""42101015800"":4}","{""421010062003"":4,""421010179005"":4,""4210102130...","{""US"":10}",3130.0,61.0,"{""<5"":1,""5-10"":1,""11-20"":0,""21-60"":5,""61-120"":...","{""Speedway"":7,""DoubleTree by Hilton"":7,""Taco B...","{""Walmart"":23,""Little Caesars"":15,""Trader Joe'...","[4,6,2,1,1,1,1,1,0,0,1,2,2,2,3,1,0,2,1,0,2,2,3,3]","{""Mo

In [79]:
cols_to_keep = ['postal_code','raw_visit_counts','normalized_visits_by_state_scaling']
padata_scale = PADATA[cols_to_keep].copy()
padata_scale.head()

,postal_code,raw_visit_counts,normalized_visits_by_state_scaling
0,19139,420.0,7287.889974
1,19136,4.0,69.408476
2,19123,15.0,260.281785
3,19116,6.0,104.112714
4,19139,35.0,607.324164


In [80]:
padata_scale1=padata_scale.groupby(['postal_code']).sum().reset_index()
padata_scale1.head()
print(len(padata_scale1))
padata_scale2 =padata_scale1.loc[padata_scale1['postal_code'].isin(zipcode_list2)]
print(len(padata_scale2))
padata_scale3 = padata_scale2.reset_index(drop=True)
padata_scale4 = pd.concat([padata_scale3]*24, ignore_index=True)
print(len(padata_scale4))
padata_scale4['scale_ratio']=padata_scale4['normalized_visits_by_state_scaling']/padata_scale4['raw_visit_counts']
padata_scale4.head()

73
48
1152


,postal_code,raw_visit_counts,normalized_visits_by_state_scaling,scale_ratio
0,19102,65273.0,1.132625e+06,17.352119
1,19103,68972.0,1.196810e+06,17.352119
2,19104,297611.0,5.164181e+06,17.352119
3,19106,61172.0,1.061464e+06,17.352119
4,19107,108520.0,1.883052e+06,17.352119


In [81]:
final_list1['state_ratio']=padata_scale4['scale_ratio']
final_list1.head()

,Unnamed: 0,postal_code,visitor_count,visitor_count_EP_POV150,visitor_count_EP_UNEMP,visitor_count_EP_NOHSDP,visitor_count_EP_AGE65,visitor_count_EP_DISABL,visitor_count_EP_MINRTY,visitor_count_EP_LIMENG,Hour,poverty,unemployed,education,aged,disabled,minority,language,ratio_wday,hour_ratio,state_ratio
0,0,19102,0.586694,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,1,366.473913,79.283763,175.813295,192.986245,203.463587,599.720139,18.435575,0.033301,0.586694,17.352119
1,1,19102,0.598095,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,2,539.362799,116.838744,314.028864,314.777500,310.786794,1201.154728,65.774159,0.033301,0.598095,17.352119
2,2,19102,0.591612,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,3,1070.245656,229.227772,582.772446,610.711860,624.227167,2219.695994,154.805481,0.033301,0.591612,17.352119
3,3,19102,0.587722,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,4,1727.136016,420.557202,899.848102,1035.934565,904.084902,3932.924253,270.573487,0.033301,0.587722,17.352119
4,4,19102,0.588930,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,5,2345.256836,567.525249,1137.192351,1350.609113,1228.436330,5271.963494,392.473580,0.033301,0.588930,17.352119


In [82]:
final_list1.to_csv('final_distribution_hourly_weekday_sampleratio_update.csv')

In [ ]:
#######residential building

In [85]:
demo_codes = ['COUNTY','FIPS','E_POV150','E_UNEMP','E_NOHSDP','E_AGE65','E_DISABL','E_MINRTY','E_LIMENG','E_TOTPOP'] #e2 is non,

# census_df_hispanic = dpf.pd_read_csv_drive(dpf.get_drive_id(filename='cbg_b03.csv', census_year=census_year), drive=drive, dtype = {'census_block_group': str})
census_df1 = pd.read_csv('svi_interactive_map.csv')
census_df1 = census_df1[demo_codes] # select only the columns we want
#census_f = census_df.rename(columns={"B01001e1":'total'})
census_df1
print(len(census_df1))
residential_rate = census_df1.loc[census_df1['COUNTY']=='Philadelphia County']
print(len(residential_rate))
residential_rate = residential_rate.reset_index(drop=True)
#residential_rate[residential_rate < 0] = 0
residential_rate.head()

3445
408


,COUNTY,FIPS,E_POV150,E_UNEMP,E_NOHSDP,E_AGE65,E_DISABL,E_MINRTY,E_LIMENG,E_TOTPOP
0,Philadelphia County,42101000101,133,86,28,242,221,531,81,1879
1,Philadelphia County,42101000102,188,109,98,277,222,455,0,2988
2,Philadelphia County,42101000200,569,82,552,298,245,2379,347,3175
3,Philadelphia County,42101000300,328,71,90,923,352,1513,112,3586
4,Philadelphia County,42101000401,672,26,104,479,359,1053,242,2577


In [86]:
zip_census = pd.read_excel('zip_census1.xlsx')

In [87]:
print(len(zip_census))
zip_census1 =zip_census.loc[zip_census['zip'].isin(zipcode_list2)]
print(len(zip_census1))
zip_census2 = zip_census1.reset_index(drop=True)
zip_census2.head()
ziplist = zip_census2.drop_duplicates(subset=['zip'])
print(len(ziplist))

651
641
48


In [88]:
residential = pd.merge(zip_census2,census_df1, left_on='census', right_on='FIPS').drop(['census','FIPS'],axis='columns')
print(len(residential))
residential.head()

595


,zip,COUNTY,E_POV150,E_UNEMP,E_NOHSDP,E_AGE65,E_DISABL,E_MINRTY,E_LIMENG,E_TOTPOP
0,19102,Philadelphia County,1073,50,56,537,549,1408,15,4072
1,19107,Philadelphia County,1073,50,56,537,549,1408,15,4072
2,19147,Philadelphia County,1073,50,56,537,549,1408,15,4072
3,19102,Philadelphia County,328,71,90,923,352,1513,112,3586
4,19103,Philadelphia County,328,71,90,923,352,1513,112,3586


In [89]:
residential1 = residential.groupby(['zip']).sum().reset_index()
print(len(residential1))
residential1.head()
residential1.to_csv('vulnerability_zip.csv')

48


In [90]:
schedule_resid = pd.read_excel('schedule_hours.xlsx')
schedule_resid.head()

,c01,c02,c03,c004,c05,c06,c07,c08,Hour,Unnamed: 9
0,0.004938,0.905171,0.008866,0.017978,0.019031,0.001705,0.032415,0.002028,1,Poverty
1,0.008801,0.847872,0.013721,0.006527,0.025440,0.001705,0.069480,0.010157,2,NaN
2,0.007867,0.681084,0.056189,0.023564,0.060773,0.001705,0.128235,0.026635,3,NaN
3,0.012947,0.466589,0.080290,0.041970,0.107788,0.019358,0.176427,0.044385,4,NaN
4,0.014272,0.301364,0.112790,0.037339,0.173793,0.061429,0.199957,0.046503,5,NaN


In [91]:
schedule_s_b.head()

,c01,c02,c03,c004,c05,c06,c07,c08,Hour,Unnamed: 9,schedule1,zipcode,schedule1_try
0,0.004938,0.905171,0.008866,0.017978,0.019031,0.001705,0.032415,0.002028,1,Poverty,0.076067,19102,0.000921
1,0.008801,0.847872,0.013721,0.006527,0.025440,0.001705,0.069480,0.010157,2,NaN,0.111952,19102,0.001641
2,0.007867,0.681084,0.056189,0.023564,0.060773,0.001705,0.128235,0.026635,3,NaN,0.222144,19102,0.001467
3,0.012947,0.466589,0.080290,0.041970,0.107788,0.019358,0.176427,0.044385,4,NaN,0.358491,19102,0.002414
4,0.014272,0.301364,0.112790,0.037339,0.173793,0.061429,0.199957,0.046503,5,NaN,0.486790,19102,0.002661


In [92]:
schedule_s_b['schedule_resid']=(schedule_s_b['c02']+schedule_s_b['c03'])
schedule_s_b.head()

,c01,c02,c03,c004,c05,c06,c07,c08,Hour,Unnamed: 9,schedule1,zipcode,schedule1_try,schedule_resid
0,0.004938,0.905171,0.008866,0.017978,0.019031,0.001705,0.032415,0.002028,1,Poverty,0.076067,19102,0.000921,0.914037
1,0.008801,0.847872,0.013721,0.006527,0.025440,0.001705,0.069480,0.010157,2,NaN,0.111952,19102,0.001641,0.861593
2,0.007867,0.681084,0.056189,0.023564,0.060773,0.001705,0.128235,0.026635,3,NaN,0.222144,19102,0.001467,0.737272
3,0.012947,0.466589,0.080290,0.041970,0.107788,0.019358,0.176427,0.044385,4,NaN,0.358491,19102,0.002414,0.546880
4,0.014272,0.301364,0.112790,0.037339,0.173793,0.061429,0.199957,0.046503,5,NaN,0.486790,19102,0.002661,0.414154


In [99]:
schedule_s_b.iloc[:1152,13]

0       0.914037
1       0.861593
2       0.737272
3       0.546880
4       0.414154
          ...   
1147    0.826327
1148    0.907564
1149    0.938519
1150    0.962898
1151    0.964053
Name: schedule_resid, Length: 1152, dtype: float64

In [93]:
print(len(schedule_s_b))

8064


In [95]:
print(len(demos_zip_m2))
residential2 = pd.concat([residential1]*24, ignore_index=True)
residential2 = residential2.sort_values(by=['zip'])
residential2 = residential2.reset_index(drop=True)

print(len(residential2))
residential2.head()

1152
1152


,zip,E_POV150,E_UNEMP,E_NOHSDP,E_AGE65,E_DISABL,E_MINRTY,E_LIMENG,E_TOTPOP
0,19102,2009,196,146,1610,1085,3462,127,9634
1,19102,2009,196,146,1610,1085,3462,127,9634
2,19102,2009,196,146,1610,1085,3462,127,9634
3,19102,2009,196,146,1610,1085,3462,127,9634
4,19102,2009,196,146,1610,1085,3462,127,9634


In [98]:
residential2.iloc[:,1]

0       2009
1       2009
2       2009
3       2009
4       2009
        ... 
1147    4982
1148    4982
1149    4982
1150    4982
1151    4982
Name: E_POV150, Length: 1152, dtype: int64

In [97]:
final_list_r = final_list1
print(len(final_list_r))
final_list_r.head()

1152


,Unnamed: 0,postal_code,visitor_count,visitor_count_EP_POV150,visitor_count_EP_UNEMP,visitor_count_EP_NOHSDP,visitor_count_EP_AGE65,visitor_count_EP_DISABL,visitor_count_EP_MINRTY,visitor_count_EP_LIMENG,Hour,poverty,unemployed,education,aged,disabled,minority,language,ratio_wday,hour_ratio,state_ratio
0,0,19102,0.586694,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,1,366.473913,79.283763,175.813295,192.986245,203.463587,599.720139,18.435575,0.033301,0.586694,17.352119
1,1,19102,0.598095,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,2,539.362799,116.838744,314.028864,314.777500,310.786794,1201.154728,65.774159,0.033301,0.598095,17.352119
2,2,19102,0.591612,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,3,1070.245656,229.227772,582.772446,610.711860,624.227167,2219.695994,154.805481,0.033301,0.591612,17.352119
3,3,19102,0.587722,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,4,1727.136016,420.557202,899.848102,1035.934565,904.084902,3932.924253,270.573487,0.033301,0.587722,17.352119
4,4,19102,0.588930,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,5,2345.256836,567.525249,1137.192351,1350.609113,1228.436330,5271.963494,392.473580,0.033301,0.588930,17.352119


In [100]:
final_list_r = final_list1
poverty_s1 = (schedule_s_b.iloc[:1152,13].values.tolist())
unemployed_s1 = (schedule_s_b.iloc[1152:2304,13].values.tolist())
education_s1 = (schedule_s_b.iloc[2304:3456,13].values.tolist())
aged_s1 = (schedule_s_b.iloc[3456:4608,13].values.tolist())
disabled_s1 = (schedule_s_b.iloc[4608:5760,13].values.tolist())
minority_s1 = (schedule_s_b.iloc[5760:6912,13].values.tolist())
language_s1 = (schedule_s_b.iloc[6912:8064,13].values.tolist())
final_list_r['poverty_r'] = residential2.iloc[:,1]*poverty_s1
final_list_r['unemployed_r'] = residential2.iloc[:,2]*unemployed_s1
final_list_r['education_r'] = residential2.iloc[:,3]*education_s1
final_list_r['aged_r'] = residential2.iloc[:,4]*aged_s1
final_list_r['disabled_r'] = residential2.iloc[:,5]*disabled_s1
final_list_r['minority_r'] = residential2.iloc[:,6]*minority_s1
final_list_r['language_r'] = residential2.iloc[:,7]*language_s1

final_list_r.head()

,Unnamed: 0,postal_code,visitor_count,visitor_count_EP_POV150,visitor_count_EP_UNEMP,visitor_count_EP_NOHSDP,visitor_count_EP_AGE65,visitor_count_EP_DISABL,visitor_count_EP_MINRTY,visitor_count_EP_LIMENG,Hour,poverty,unemployed,education,aged,disabled,minority,language,ratio_wday,hour_ratio,state_ratio,poverty_r,unemployed_r,education_r,aged_r,disabled_r,minority_r,language_r
0,0,19102,0.586694,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,1,366.473913,79.283763,175.813295,192.986245,203.463587,599.720139,18.435575,0.033301,0.586694,17.352119,1836.300322,187.841833,135.193213,1466.631310,1005.175807,3120.293341,118.103773
1,1,19102,0.598095,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,2,539.362799,116.838744,314.028864,314.777500,310.786794,1201.154728,65.774159,0.033301,0.598095,17.352119,1730.940491,169.327187,122.335197,1392.820413,950.046780,2852.565018,113.691085
2,2,19102,0.591612,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,3,1070.245656,229.227772,582.772446,610.711860,624.227167,2219.695994,154.805481,0.033301,0.591612,17.352119,1481.180398,140.251317,103.026198,1148.860491,794.067370,2314.539702,98.027191
3,3,19102,0.587722,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,4,1727.136016,420.557202,899.848102,1035.934565,904.084902,3932.924253,270.573487,0.033301,0.587722,17.352119,1098.681575,102.488061,72.418013,937.334607,604.716179,1722.894901,75.900933
4,4,19102,0.588930,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,5,2345.256836,567.525249,1137.192351,1350.609113,1228.436330,5271.963494,392.473580,0.033301,0.588930,17.352119,832.034869,66.554169,49.922498,704.019809,452.899490,1332.551466,59.559331


In [101]:

final_list_r['poverty_c'] = (final_list_r['poverty']*final_list_r['ratio_wday']*final_list_r['hour_ratio']*final_list_r['state_ratio'])
final_list_r['unemployed_c'] = (final_list_r['unemployed']*final_list_r['ratio_wday']*final_list_r['hour_ratio']*final_list_r['state_ratio'])
final_list_r['education_c'] = (final_list_r['education']*final_list_r['ratio_wday']*final_list_r['hour_ratio']*final_list_r['state_ratio'])
final_list_r['aged_c'] = (final_list_r['aged']*final_list_r['ratio_wday']*final_list_r['hour_ratio']*final_list_r['state_ratio'])
final_list_r['disabled_c'] = (final_list_r['disabled']*final_list_r['ratio_wday']*final_list_r['hour_ratio']*final_list_r['state_ratio'])
final_list_r['minority_c'] = (final_list_r['minority']*final_list_r['ratio_wday']*final_list_r['hour_ratio']*final_list_r['state_ratio'])
final_list_r['language_c'] = (final_list_r['language']*final_list_r['ratio_wday']*final_list_r['hour_ratio']*final_list_r['state_ratio'])

final_list_r['poverty_all'] = (final_list_r['poverty']*final_list_r['ratio_wday']*final_list_r['hour_ratio']*final_list_r['state_ratio'])+final_list_r['poverty_r']
final_list_r['unemployed_all'] = (final_list_r['unemployed']*final_list_r['ratio_wday']*final_list_r['hour_ratio']*final_list_r['state_ratio'])+final_list_r['unemployed_r']
final_list_r['education_all'] = (final_list_r['education']*final_list_r['ratio_wday']*final_list_r['hour_ratio']*final_list_r['state_ratio'])+final_list_r['education_r']
final_list_r['aged_all'] = (final_list_r['aged']*final_list_r['ratio_wday']*final_list_r['hour_ratio']*final_list_r['state_ratio'])+final_list_r['aged_r']
final_list_r['disabled_all'] = (final_list_r['disabled']*final_list_r['ratio_wday']*final_list_r['hour_ratio']*final_list_r['state_ratio'])+final_list_r['disabled_r']
final_list_r['minority_all'] = (final_list_r['minority']*final_list_r['ratio_wday']*final_list_r['hour_ratio']*final_list_r['state_ratio'])+final_list_r['minority_r']
final_list_r['language_all'] = (final_list_r['language']*final_list_r['ratio_wday']*final_list_r['hour_ratio']*final_list_r['state_ratio'])+final_list_r['language_r']
final_list_r.head()

,Unnamed: 0,postal_code,visitor_count,visitor_count_EP_POV150,visitor_count_EP_UNEMP,visitor_count_EP_NOHSDP,visitor_count_EP_AGE65,visitor_count_EP_DISABL,visitor_count_EP_MINRTY,visitor_count_EP_LIMENG,Hour,poverty,unemployed,education,aged,disabled,minority,language,ratio_wday,hour_ratio,state_ratio,poverty_r,unemployed_r,education_r,aged_r,disabled_r,minority_r,language_r,poverty_c,unemployed_c,education_c,aged_c,disabled_c,minority_c,language_c,poverty_all,unemployed_all,education_all,aged_all,disabled_all,minority_all,language_all
0,0,19102,0.586694,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,1,366.473913,79.283763,175.813295,192.986245,203.463587,599.720139,18.435575,0.033301,0.586694,17.352119,1836.300322,187.841833,135.193213,1466.631310,1005.175807,3120.293341,118.103773,124.239763,26.878300,59.603156,65.425026,68.976991,203.313483,6.249917,1960.540084,214.720133,194.796370,1532.056336,1074.152798,3323.606824,124.353690
1,1,19102,0.598095,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,2,539.362799,116.838744,314.028864,314.777500,310.786794,1201.154728,65.774159,0.033301,0.598095,17.352119,1730.940491,169.327187,122.335197,1392.820413,950.046780,2852.565018,113.691085,186.404859,40.379703,108.529002,108.787732,107.408536,415.121470,22.731681,1917.345350,209.706890,230.864199,1501.608145,1057.455316,3267.686489,136.422766
2,2,19102,0.591612,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,3,1070.245656,229.227772,582.772446,610.711860,624.227167,2219.695994,154.805481,0.033301,0.591612,17.352119,1481.180398,140.251317,103.026198,1148.860491,794.067370,2314.539702,98.027191,365.869730,78.362853,199.224165,208.775417,213.395704,758.816043,52.921158,1847.050128,218.614169,302.250363,1357.635909,1007.463074,3073.355746,150.948349
3,3,19102,0.587722,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,4,1727.136016,420.557202,899.848102,1035.934565,904.084902,3932.924253,270.573487,0.033301,0.587722,17.352119,1098.681575,102.488061,72.418013,937.334607,604.716179,1722.894901,75.900933,586.549468,142.824653,305.595749,351.811821,307.034601,1335.653130,91.888961,1685.231042,245.312714,378.013762,1289.146429,911.750780,3058.548031,167.789895
4,4,19102,0.588930,4817.796,1445.848,2167.974,3294.7,2905.86,9034.371,673.294,5,2345.256836,567.525249,1137.192351,1350.609113,1228.436330,5271.963494,392.473580,0.033301,0.588930,17.352119,832.034869,66.554169,49.922498,704.019809,452.899490,1332.551466,59.559331,798.104292,193.132083,386.993050,459.619993,418.043897,1794.079278,133.561000,1630.139161,259.686252,436.915548,1163.639802,870.943387,3126.630744,193.120331


In [102]:
final_list_r.to_csv('final_result_update0118.csv')
